In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from tensorflow import keras
import tensorflow as tf

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler,ReduceLROnPlateau

from tensorflow.keras import layers

import warnings
warnings.filterwarnings("ignore")

In [14]:
img_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.2,
    rotation_range=5,
    shear_range=0.3,
    zoom_range=0.3,
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip=True,
    vertical_flip=True,
)

img_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.2,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=(0.9,1.1)
)

In [15]:
train_ds = img_datagen.flow_from_directory('/kaggle/input/4-animal-classification/train',subset="training", class_mode = 'categorical', batch_size=16, target_size=(224,224))
val_ds = img_datagen.flow_from_directory('/kaggle/input/4-animal-classification/train',subset="validation", class_mode = 'categorical', batch_size=16, target_size=(224,224))

Found 2240 images belonging to 4 classes.
Found 560 images belonging to 4 classes.


In [16]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

(16, 224, 224, 3)
(16, 4)


In [10]:
model = tf.keras.Sequential()

model.add(layers.Conv2D(256, (3,3), activation='relu', input_shape=(224,224,3)))
model.add(layers.MaxPooling2D(2, 2))

model.add(layers.Conv2D(128, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(128, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Dropout(0.3))

model.add(layers.Conv2D(64, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)))
model.add(layers.BatchNormalization())

model.add(layers.MaxPooling2D(2,2))

model.add(layers.Conv2D(32, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(2,2))

model.add(layers.Flatten())

model.add(layers.Dense(2048, activation='relu'))
model.add(layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 256)     7168      
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 256)     0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 128)     295040    
                                                                 
 batch_normalization (Batch  (None, 109, 109, 128)     512       
 Normalization)                                                  
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 128)       0         
 g2D)                                                            
                                                        

In [12]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy',factor=0.8, patience=10, verbose=1)
save_best = tf.keras.callbacks.ModelCheckpoint("Model.h5",monitor='val_accuracy',save_best_only=True, verbose=1)

In [14]:
model.fit(train_ds, validation_data=val_ds, epochs=5, callbacks=[save_best], shuffle=True)

Epoch 1/5
124/140 [=========================>....] - ETA: 1:36 - loss: 3.1133 - accuracy: 0.3649
Epoch 1: val_accuracy improved from -inf to 0.21429, saving model to Model.h5
140/140 [==============================] - 909s 6s/step - loss: 3.0995 - accuracy: 0.3750 - val_loss: 3.9421 - val_accuracy: 0.2143
Epoch 2/5
140/140 [==============================] - ETA: 0s - loss: 2.9152 - accuracy: 0.4554
Epoch 2: val_accuracy did not improve from 0.21429
140/140 [==============================] - 887s 6s/step - loss: 2.9152 - accuracy: 0.4554 - val_loss: 5.1434 - val_accuracy: 0.2143
Epoch 3/5
140/140 [==============================] - ETA: 0s - loss: 2.7977 - accuracy: 0.5009
Epoch 3: val_accuracy did not improve from 0.21429
140/140 [==============================] - 885s 6s/step - loss: 2.7977 - accuracy: 0.5009 - val_loss: 4.2382 - val_accuracy: 0.2143
Epoch 4/5
140/140 [==============================] - ETA: 0s - loss: 2.7193 - accuracy: 0.4924
Epoch 4: val_accuracy improved from 0.2142

In [15]:
model.evaluate(val_ds)

35/35 [==============================] - 61s 2s/step - loss: 2.7680 - accuracy: 0.4679


[2.767965078353882, 0.46785715222358704]

# CNN : loss-2.767965078353882, Accuracy- 0.46785715222358704

In [5]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image

In [ ]:
base_model = tf.keras.applications.Xception(weights='imagenet', input_shape=(224,224,3), include_top=False)
base_model.trainable = False

In [6]:
weights_path = '/kaggle/input/vgg16-model/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'


base_model= VGG16(weights=weights_path, input_shape=(224,224,3), include_top=False)
base_model.trainable = False

In [9]:
model1 = tf.keras.Sequential()

model1.add(base_model)
model1.add(layers.BatchNormalization())

model1.add(layers.Conv2D(128, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)))
model1.add(layers.Dropout(0.5))

model1.add(layers.GlobalAveragePooling2D())
model1.add(layers.Dropout(0.5))

model1.add(layers.Dense(4096, activation='relu'))
model1.add(layers.Dropout(0.5))
model1.add(layers.Dense(2048, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)))
model1.add(layers.Dropout(0.5))
model1.add(layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)))
model1.add(layers.Dropout(0.5))
model1.add(layers.Dense(4, activation='softmax'))



In [10]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 batch_normalization (Batch  (None, 7, 7, 512)         2048      
 Normalization)                                                  
                                                                 
 conv2d (Conv2D)             (None, 5, 5, 128)         589952    
                                                                 
 dropout (Dropout)           (None, 5, 5, 128)         0         
                                                                 
 global_average_pooling2d (  (None, 128)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dropout_1 (Dropout)         (None, 128)               0

In [17]:
model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy',factor=0.8, patience=10, verbose=1)
save_best = tf.keras.callbacks.ModelCheckpoint("Model-vgg16.h5",monitor='val_accuracy',save_best_only=True, verbose=1)

In [19]:
model1.fit(train_ds, validation_data=val_ds, epochs=5, callbacks=[save_best], shuffle=True)

Epoch 1/5
140/140 [==============================] - ETA: 0s - loss: 5.4028 - accuracy: 0.3777
Epoch 1: val_accuracy improved from -inf to 0.66429, saving model to Model-vgg16.h5
140/140 [==============================] - 827s 6s/step - loss: 5.4028 - accuracy: 0.3777 - val_loss: 5.0780 - val_accuracy: 0.6643
Epoch 2/5
140/140 [==============================] - ETA: 0s - loss: 4.7501 - accuracy: 0.6170
Epoch 2: val_accuracy improved from 0.66429 to 0.74286, saving model to Model-vgg16.h5
140/140 [==============================] - 824s 6s/step - loss: 4.7501 - accuracy: 0.6170 - val_loss: 4.4315 - val_accuracy: 0.7429
Epoch 3/5
140/140 [==============================] - ETA: 0s - loss: 4.3127 - accuracy: 0.7277
Epoch 3: val_accuracy improved from 0.74286 to 0.76250, saving model to Model-vgg16.h5
140/140 [==============================] - 866s 6s/step - loss: 4.3127 - accuracy: 0.7277 - val_loss: 4.1141 - val_accuracy: 0.7625
Epoch 4/5
140/140 [==============================] - ETA: 0s 

In [21]:
%%time
model1.evaluate(val_ds)

35/35 [==============================] - 160s 5s/step - loss: 3.5904 - accuracy: 0.8071
CPU times: user 9min 46s, sys: 27.3 s, total: 10min 13s
Wall time: 2min 40s


[3.5903587341308594, 0.8071428537368774]

# VGG16 : loss-3.5903587341308594, Accuracy-0.8071428537368774

In [46]:
base_model2 = tf.keras.applications.Xception(weights='/kaggle/input/xception-model/xception_weights_tf_dim_ordering_tf_kernels_notop.h5', input_shape=(224,224,3), include_top=False)
base_model2.trainable = False

In [47]:
%%time
model2 = tf.keras.Sequential()

model2.add(base_model2)
model2.add(layers.BatchNormalization())

model2.add(layers.Conv2D(128, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)))
model2.add(layers.Dropout(0.5))

model2.add(layers.GlobalAveragePooling2D())
model2.add(layers.Dropout(0.5))

model2.add(layers.Dense(4096, activation='relu'))
model2.add(layers.Dropout(0.5))
model2.add(layers.Dense(2048, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)))
model2.add(layers.Dropout(0.5))
model2.add(layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)))
model2.add(layers.Dropout(0.5))
model2.add(layers.Dense(4, activation='softmax'))


CPU times: user 879 ms, sys: 151 ms, total: 1.03 s
Wall time: 798 ms


In [48]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 batch_normalization_9 (Bat  (None, 7, 7, 2048)        8192      
 chNormalization)                                                
                                                                 
 conv2d_9 (Conv2D)           (None, 5, 5, 128)         2359424   
                                                                 
 dropout_5 (Dropout)         (None, 5, 5, 128)         0         
                                                                 
 global_average_pooling2d_1  (None, 128)               0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dropout_6 (Dropout)         (None, 128)              

In [49]:
model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

In [50]:
lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy',factor=0.8, patience=10, verbose=1)
save_best1 = tf.keras.callbacks.ModelCheckpoint("Model-Xception.h5",monitor='val_accuracy',save_best_only=True, verbose=1)

In [51]:
model2.fit(train_ds, validation_data=val_ds, epochs=5, callbacks=[save_best1], shuffle=True)

Epoch 1/5
140/140 [==============================] - ETA: 0s - loss: 4.7761 - accuracy: 0.7808
Epoch 1: val_accuracy improved from -inf to 0.96429, saving model to Model-Xception.h5
140/140 [==============================] - 365s 3s/step - loss: 4.7761 - accuracy: 0.7808 - val_loss: 4.2514 - val_accuracy: 0.9643
Epoch 2/5
140/140 [==============================] - ETA: 0s - loss: 4.2588 - accuracy: 0.9348
Epoch 2: val_accuracy did not improve from 0.96429
140/140 [==============================] - 359s 3s/step - loss: 4.2588 - accuracy: 0.9348 - val_loss: 4.1329 - val_accuracy: 0.9482
Epoch 3/5
140/140 [==============================] - ETA: 0s - loss: 3.8580 - accuracy: 0.9375
Epoch 4: val_accuracy did not improve from 0.96429
140/140 [==============================] - 359s 3s/step - loss: 3.8580 - accuracy: 0.9375 - val_loss: 3.7497 - val_accuracy: 0.9554
Epoch 5/5
140/140 [==============================] - ETA: 0s - loss: 3.6427 - accuracy: 0.9580
Epoch 5: val_accuracy improved from

In [52]:
model2.evaluate(val_ds)

35/35 [==============================] - 65s 2s/step - loss: 3.5925 - accuracy: 0.9625


[3.5925071239471436, 0.9624999761581421]

# Xception : loss-3.5925071239471436, Accuracy- 0.9624999761581421

In [53]:
mm = tf.keras.models.load_model('/kaggle/working/Model-Xception.h5')

In [63]:
test_path = '/kaggle/input/4-animal-classification/test'
test_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255).flow_from_directory(    
    directory=test_path,
    target_size=(224, 224),
    batch_size=16,
    classes=['.'],
    shuffle=False,
)

Found 729 images belonging to 1 classes.


In [58]:
from PIL import Image
predict = []

In [69]:


for i in range(1, 730):
    img = Image.open('/kaggle/input/4-animal-classification/test/test/' + str(i) + '.jpg')
    img = img.resize((224, 224))
    img = np.asarray(img) / 255.0
    c_predict = model2.predict(np.asarray(img).reshape(-1, 224, 224, 3))
    predict.append(c_predict)

    if i % 100 == 0:
        print(i, "Predicted")


1/1 [==============================] - 0s 157ms/step
100 Predicted
1/1 [==============================] - 0s 164ms/step
200 Predicted
1/1 [==============================] - 0s 163ms/step
300 Predicted
1/1 [==============================] - 0s 165ms/step
400 Predicted
1/1 [==============================] - 0s 163ms/step
500 Predicted
1/1 [==============================] - 0s 166ms/step
600 Predicted
1/1 [==============================] - 0s 171ms/step
700 Predicted
1/1 [==============================] - 0s 159ms/step


In [121]:
predictions = np.argmax(predict,axis=2)
pp = predictions.reshape(-1)

print(len(ids))
len(pp)


1494


1494

In [123]:
ids = [i for i in range(1,1495)]
label = pp
subb = pd.DataFrame({'ID':ids,'Label':pp})

subb.to_csv('submission.csv', index=False)


In [125]:
subb['Label'].value_counts()

Label
2    474
0    407
3    352
1    261
Name: count, dtype: int64